# Python to JavaScript tips

### Print is a jupyter demonstration function only -- not for pavlovia

In [1]:
%%HTML
<script>
current_cell = undefined;
function print(...value) {
    //# get the currently running cell:
    var outers = document.getElementsByClassName("cell code_cell rendered running")[0];
    //# the area into which text will be printed:
    var print_space = outers.getElementsByClassName("output_subarea output_html rendered_html")[0];
    //# make the current cell empty the first time you print to it.
    if (!current_cell || current_cell != print_space){
        current_cell = print_space;
        print_space.innerHTML = "";
    }
    //# add the text to a new line with div:
    print_space.innerHTML += ("<div>"+value+"</div>").replaceAll(", ", ",").replaceAll(",", ", ").replaceAll("\n", "</div><div>");
    //print_space.innerHTML = "";
    
    
}
print("Hello Jupyter Cell");
</script>

# PsychoPy to Pavlovia conversions made easy

## Helper functions

In [2]:
%%HTML
<script>
//# because Pavlovia doesn't auto convert append or extend to push, 
//#  make the same functions work for any javascript array by affecting the prototype
Array.prototype.append = Array.prototype.push
Array.prototype.extend = function(arr) {this.push(...arr)};

arr = [1,2,3,4]
arr.append(5)
print(arr)
arr.extend([6,7,8])
print(arr)
</script>

In [33]:
%%HTML
<script>
//# how to format a class so that Pavlovia will accept it.
//#  can convert the quest plus class to this 
//#  format to make it work in Pavlovia without pasting it after export.
A = class {
    constructor(name) {
        this.name = name;
    }
    func(replace_name) {
        this.name = replace_name;
    }
}

a = new A("my name");
print(a.name)
a.func("new name");
print(a.name)
</script>

In [3]:
%%HTML
<script>
//# avoid using the "function name(){}" syntax
//#  to account for Pavlovia's scope problems.
range = function(start_stop, post_stop, step=1) {
    //# create a list sequence with a starting point, stopping point, and step size
    //#  the step size is assumed 1 if not provided
    //#  the first value is considered the stopping point if the second value is not given,
    //#   in which case the starting point is assumed to be 0.
    //#  post_stop is the non-inclusive end point.
    var arr = [];
    if (post_stop != undefined) {
        for (var i = start_stop; i < post_stop; i+=step) {
            arr.push(i);
        }
    } else {
        for (var i = 0; i < start_stop; i+=step) {
            arr.push(i);
        }
    }
    return arr;
}

round = function(value, decimals) {
    //# JavaScript's Math.round() doesn't normally
    //#  allow you to specify how many decimals you want!
    var tmp = Math.pow(10, decimals);
    return Math.round(value*tmp)/tmp;
}

arange = function(start_stop, post_stop, step=1, rounding=2) {
    //# same as range(), but with rounding built-in
    //# create a list sequence with a starting point, stopping point, and step size
    //#  the step size is assumed 1 if not provided
    //#  the first value is considered the stopping point if the second value is not given,
    //#   in which case the starting point is assumed to be 0.
    //#  post_stop is the non-inclusive end point.
    var arr = [];
    if (post_stop != undefined) {
        for (var i = start_stop; i < post_stop; i+=step) {
            arr.push(round(i,rounding));
        }
    } else {
        for (var i = 0; i < start_stop; i+=step) {
            arr.push(round(i,rounding));
        }
    }
    return arr;
}

clone = function(arr) {
    //# a recursive function that deep copies arrays and dictionaries    
    var loops, tmp;
    //# see if it is an array
    //#  check a couple of things to try to avoid objects with these keys
    if (Array.isArray(arr)) {
        loops = arr;
        tmp = [];
    } else {
        loops = Object.keys(arr);
        tmp = {};
    }
    loops.forEach(function(val){
        var value;
        //# if the looped items are the array, push() to a list
        if (loops === arr) {
            value = val;
            if (typeof(value) == "object") {
                //# if the subelement is another complex object, 
                //#  then clone it as well.
                tmp.push(clone(value));
            } else {
                tmp.push(value);
            }
        } else { //# if the looped items are the dictionary
                 //#  add them with the key
            value = arr[val];
            if (typeof(value) == "object") {
                //# if the subelement is another complex object, 
                //#  then clone it as well.
                tmp[val] = clone(value);
            } else {
                tmp[val] = value;
            }
        }
    })
    return tmp;
}
//# assumes start of 0 and step size of 1,
//#  and ends before 10
print("assumed start: 0, stop: 10, assumed step: 1")
print(range(10));
//# start of 5, end of 10, but assumes step size of 1.
print("start: 5, stop: 10, assumed step: 1")
print(range(5,10));
//# start of 0, end of 10, step size of 3.
print("start: 0, stop: 10, step: 3")
print(range(0,10,3));
//# includes float support, but doesn't 
//#  round or have as high floating point precision 
//#  like python's np.arange()
print("Floating point steps")
print(range(0,10.01,.5));
print(range(0,1.01,.1));
print("Rounded floating point step")
print(arange(0,1.01,.05, 2));

</script>

In [4]:
%%HTML
<script>
//# contains useful random number generation functions.
random = {"random": function(start_stop=1, post_stop=undefined, count=1) {
                var results = [];
                range(count).forEach(function(){
                  //# Math.random() never gives exactly 1, so it
                  //#  never reaches the provided max value.
                  if (post_stop != undefined) {
                      if (post_stop < start_stop) {
                          var tmp = post_stop;
                          post_stop = start_stop;
                          start_stop = tmp;
                      }
                      results.push((start_stop+Math.random()*(post_stop-start_stop)));
                  } else {
                      //# if post_stop is not defined, start_stop is the non-inclusive max value.
                      results.push((Math.random()*start_stop));
                  }
                });
                if (count == 1)
                    return results[0];
                return results;       
          }, 
          "randint":function(start_stop, post_stop, count=1){
                var results = [];
                range(count).forEach(function(){
                  //# Math.random() never gives exactly 1, so it
                  //#  never reaches the provided max value.
                  if (post_stop != undefined) {
                      if (post_stop < start_stop) {
                          var tmp = post_stop;
                          post_stop = start_stop;
                          start_stop = tmp;
                      }
                      results.push(parseInt(start_stop+Math.random()*(post_stop-start_stop)));
                  } else {
                      //# if post_stop is not defined, start_stop is the non-inclusive max value.
                      results.push(parseInt(Math.random()*start_stop));
                  }
                });
                if (count == 1)
                    return results[0];
                return results;
          },
          "choice":function(arr, count=1, with_replacement=true){
                var results = [];
                if (with_replacement) {
                    range(count).forEach(function(){
                        results.push(arr[random.randint(arr.length)]);
                    });
                } else {
                    //# make a shallow copy to choose from.
                    //#  you can always clone the returned result if you want unique copies,
                    //#  and it makes no difference for arrays of primitives 
                    //#  (like numeric indices or string keys).
                    var tmp = [...arr];//clone(arr);
                    console.log(tmp);
                    range(count).forEach(function(){
                        if (tmp.length == 0)
                            //# refill the array
                            tmp = [...arr];//clone(arr);
                        results.push(tmp.splice(random.randint(tmp.length),1));
                    });
                }
                if (count == 1)
                    return results[0];
                return results;
          }};
arr = [1,2,3,4,5];
print("5 Random floats in the space [0,1)")
print(random.random(0,1,5));
print("5 Random floats in the space [5,10)")
print(random.random(5,10,5));
print("5 Random integers in the space [5,10)")
print(random.randint(5,10, 5));
print("5 Random choices with replacement")
print(random.choice(arr, 5));
print("5 Random choices without replacement")
print(random.choice(arr, 5, false));
//# causes repeats only after the array is depleted.
print("Random choices without replacement, exceeding the size of the array")
print(random.choice(arr, arr.length*2, false));
</script>

## For Each Loops
JavaScript's for loops are usually very different than Python's, but there are ways to make them do very similar things.

In [5]:
# loops through list/array contents

# python's range function is a generator, rather than a list
arr = list(range(1,6))
for value in arr:
    print(value)

# same as:
for i in range(len(arr)):
    print(arr[i])

1
2
3
4
5
1
2
3
4
5


In [6]:
%%HTML
<script>
arr = range(1,6);
//# this JavaScript for loop version looks silly, but has 
//#  fewer places to make mistakes, and
//#  as we will see later, is the more generically useful of the two forms.
arr.forEach(function(value) {
    print(value);
});

//# same as above, but more settings:
//# there are controls for the starting point; stopping point; and step size (i++ is the same as i+=1)
for (var i=0; i < arr.length; i++) {
    print(arr[i]);
}
</script>

# Scope and complex .forEach instances

In [38]:
# loops through list/array contents while also enumerating to get the index
arr = list(range(1,6))
for i,value in enumerate(arr):
    print("index:", i, " value:", value, " arbitrary indexing:", arr[0])

# the "scope" refers to the space in which a variable is declared.
#  indentations past each ':' for functions, conditionals, and loops are different scopes 
#  and can declare separate variables with the same names without influencing each other.

def scope_demoA():
    # JavaScript assumes a variable is global unless told otherwise.
    # Python assumes a variable is local unless told otherwise.
    
    # by default, a variable is local to this Python function 
    #  and not affect the outer, global arr:
    arr = [4,3,2,1]
    return arr

def scope_demoB():
    # this is how you make a variable global to this Python function 
    #  and affect the outer, global arr:
    global arr # this must be the first real line of code in the function!
    
    arr = [4,3,2,1]
    return arr

print("Locally change Arr")
print(scope_demoA())
print("Unchanged:")
print(arr)
print("Globally change Arr")
print(scope_demoB())
print("Changed:")
print(arr)

index: 0  value: 1  arbitrary indexing: 1
index: 1  value: 2  arbitrary indexing: 1
index: 2  value: 3  arbitrary indexing: 1
index: 3  value: 4  arbitrary indexing: 1
index: 4  value: 5  arbitrary indexing: 1
Locally change Arr
[4, 3, 2, 1]
Unchanged:
[1, 2, 3, 4, 5]
Globally change Arr
[4, 3, 2, 1]
Changed:
[4, 3, 2, 1]


In [8]:
%%HTML
<script>
arr = range(1,6);
arr.forEach(function(value, i) {
    print("index: " + i, " value: " + value, " arbitrary indexing: " + arr[0])
});
print("Strict scope pedantry")
//# the previous scope technially already defines arr, 
//#  but there is an optional 3rd parameter you can declare it explicitly
//#  if you have very complex code with many scopes.

//# the "scope" refers to the space in which a variable is declared.
//#  each set of {}'s for functions, conditionals, and loops are different scopes 
//#  and can declare separate variables with the same names without influencing each other.


//#  Pavlovia handles declarations of global variables in the background, 
//#  so only declare variables in builder with "var" if you are using it in a function where it should be separate.
arr.forEach(function(value, i, arr_scope_name) {
    print("index: " + i, " value: " + value, " arbitrary indexing: " + arr_scope_name[0])
});

function scope_demoA(){
    //# JavaScript assumes a variable is global unless told otherwise.
    //# Python assumes a variable is local unless told otherwise.
    
    //# this is how you make a variable local to this JavaScript function 
    //#  and not affect the outer, global arr:
    var arr = [4,3,2,1];
    return arr;
}
function scope_demoB(){
    arr = [5,4,3,2];
    return arr;
}
//# scope demo A's "arr" doesn't override the global arr
print("Locally change Arr")
print(scope_demoA());
print("Unchanged:")
print(arr);

//# scope demo B's "arr" does override the global arr
print("Globally change Arr")
print(scope_demoB());
print("Changed:")
print(arr);

</script>

# Dictionaries

In [40]:
d = {'a':1, 'b':2, 'c':3}
for key in d:
    # do something with each key, including accessing the dictionary.
    print(key+",", d[key])

a, 1
b, 2
c, 3


In [10]:
%%HTML
<script>
//# dictionary's rough equivalent is an Object:
d = {'a':1, 'b':2, 'c':3}
//# get the list of dictionary keys from the Object named 'd'
keys = Object.keys(d);
//# do something with each key, including accessing the dictionary.
keys.forEach(function(key) {
    print(key, " " + d[key]);
});
</script>

# List Comprehension

In [11]:
# the minimum requirements of list comprehension: construct an array based on 
#  an expression that gives a value (left side) and a loop (right side).
p = [i * i for i in range(1, 10)]
print("The resulting list from a list comprehension containing squared values from 1 to 9")
print(p)

The resulting list from a list comprehension containing squared values from 1 to 9
[1, 4, 9, 16, 25, 36, 49, 64, 81]


In [ ]:
%%HTML
<script>
// make the empty array to build into.
p = [];
// add values to the array with the push() function.
range(1, 10).forEach(function(i){p.push(i * i)});
print("The resulting list from a list comprehension containing squared values from 1 to 9")
print(p);

</script>

In [12]:
%%HTML
<script>
// make the empty array to build into.
preexisting = random.randint(1, 51, 72);
alpha = 1.1;
beta = 0.5;
prob = 0.5;
amb = 0;
sv = [];
// add values to the array with the push() function.
preexisting.forEach(function(money){sv.push((prob - beta*amb/2)*Math.pow(money, alpha))});
print("The resulting list from a list comprehension containing squared values from 1 to 9")
print(sv);

</script>

# List Comprehension w/ Conditionals

In [44]:
# conditional list comprehension adds a conditional statement to 
# whether the expression should be added to the new list.
p = [i * i for i in range(1, 10) if i%2==0]
print("Only even values squared and kept in a list comprehension")
print(p)

Only even values squared and kept in a list comprehension
[4, 16, 36, 64]


In [13]:
%%HTML
<script>
//# make the empty array to build into.
p = [];
//# if there is exactly one line 'underneath' it 
//#  (like in this example and all python list comprehensions), 
//#  then the if statement does not need to add extra brackets.
//# add values to the array with the push() function.
range(1, 10).forEach(function(i){if(i%2==0) p.push(i * i)});
print("Only even values squared and kept in a list comprehension")
print(p);

</script>

## Copying 1-dimensional arrays of primitives

In [14]:
arr = [1,2,3,4,5]
# lists and dictionaries are stored by the _reference_ 
#  to the real content in memory, 
#  so errors like this don't make copies of the contents, 
#  just the reference to the same content.
bad_copy = arr
# if the contents are primitive (like numbers and strings)
#  you can place them all into a new list with the unpacking operator '*'
arr_copy = [*arr]
arr_copy[0] = 5

print("Correctly copied contents modified separately:")
print(arr_copy)
print("Original contents not modified by a good copy:")
print(arr)
# using copies of a reference affects the original contents
bad_copy[0] = 0
print("Original contents modified by a bad copy.")
print(arr)

Correctly copied contents modified separately:
[5, 2, 3, 4, 5]
Original contents not modified by a good copy:
[1, 2, 3, 4, 5]
Original contents modified by a bad copy.
[0, 2, 3, 4, 5]


In [15]:
%%HTML
<script>
arr = [1,2,3,4,5];
bad_copy = arr;
//# the main difference in JavaScript is that the unpacking operator is different.
//# It's "..." instead of "*"
arr_copy = [...arr];
arr_copy[0] = 5;

print("Correctly copied contents modified separately:")
print(arr_copy)
print("Original contents not modified by a good copy:")
print(arr)
//# using copies of a reference still affects the original contents
bad_copy[0] = 0
print("Original contents modified by a bad copy.")
print(arr)
</script>

## Copying multidimensional arrays

In [17]:
# python can use numpy to copy things, but it needs 
#  to be a numpy array and not a standard python list.
import numpy as np
arr = np.array([list(range(1,10,2)), list(range(2,12,2))])
arr_copy = np.ndarray.copy(arr)
arr_copy[0,1] = -10
print("Copy")
print(arr_copy)
print("Original")
print(arr)

# again, demonstrating copying just the reference to the content in memory is bad.
bad_copy = arr
bad_copy[0,1] = 999
print("Original Contents modified by a bad copy.")
print(arr)

# you can convert it back to a python list though:
print("From numpy back to List form.")
print(arr_copy.tolist())

Copy
[[  1 -10   5   7   9]
 [  2   4   6   8  10]]
Original
[[ 1  3  5  7  9]
 [ 2  4  6  8 10]]
Original Contents modified by a bad copy.
[[  1 999   5   7   9]
 [  2   4   6   8  10]]
From numpy back to List form.
[[1, -10, 5, 7, 9], [2, 4, 6, 8, 10]]


In [31]:
%%HTML
<script>

arr = [range(1,10,2), range(2,12,2), 3];
arr_copy = clone(arr);
//# can't use tuple indexing like in python
//#  must use a pair of []'s for every dimension
arr_copy[0][1] = -10;
arr_copy[2] = 1
//# my print is simpler than python's, 
//#  so let's separate the dimensions with two prints

print("Original Contents");
print(arr[0])
print(arr[1])
print(arr[2])

print("Good Copy Contents");
print(arr_copy[0])
print(arr_copy[1])
print(arr_copy[2])

print("Original Contents -- still the same");
print(arr[0])
print(arr[1])
print(arr[2])


//# again, demonstrating copying just the reference to the content in memory is bad.
bad_copy = [...arr];
bad_copy[0][0] = 999;
bad_copy[2] = 1;
print("Bad Copy Contents");
print(bad_copy[0])
print(bad_copy[1])
print(bad_copy[2])
print("Original Contents modified by a bad (shallow) copy.");
print(arr[0])
print(arr[1])
print(arr[2])
</script>

## Cloning dictionaries

In [50]:
import pandas as pd
d = {"value":list(range(10)), "category":list(range(10))}
d_copy = pd.DataFrame(d).copy().to_dict('list')
d_copy["value"][0] = -10
print("Copy modified separately", d_copy)
print("Unmodified original     ", d)
# again, demonstrating copying just the reference to the content in memory is bad.
bad_copy = d;
bad_copy["value"][0] = 999
print("Original Contents modified by a bad copy:\n                        ", d)

Copy modified separately {'value': [-10, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'category': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}
Unmodified original      {'value': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'category': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}
Original Contents modified by a bad copy:
                         {'value': [999, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'category': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}


In [33]:
%%HTML
<script>
//# the provided clone() also works for dictionaries with arrays.
d = {"category":range(10), "value":range(10)}
d_copy = clone(d);
d_copy["value"][0] = -1;
print("Copied Contents\n" + d_copy["value"])
print("Original Contents\n" + d["value"])


//# again, demonstrating copying just the reference to the content in memory is bad.
bad_copy = d;
bad_copy["value"][0] = 999
print("Original Contents modified by a bad copy.\n" + d["value"])


</script>

In [ ]:
%%HTML
<script>
//# risk and ambiguity model
function SV_option_RA(stimulus_values, params) {
    var objective_value = stimulus_values[0];
    var ambiguity = stimulus_values[1];
    var probability = stimulus_values[2];
    var value_certain = stimulus_values[3];
    var alpha_subject_risk_aversion = params[0];
    var beta_ambiguity_aversion = params[1];
    
    var sv_reward, sv_null;
    sv_reward = Math.sign(objective_value)*(probability - beta_ambiguity_aversion * (ambiguity/2)) * Math.pow(Math.abs(objective_value), alpha_subject_risk_aversion);
    sv_null = Math.sign(value_certain)*Math.pow(Math.abs(value_certain), alpha_subject_risk_aversion);

    var gamma = params[2];
    // probability of selecting the lottery
    prob = 1. / (1 + Math.exp(-gamma * (sv_reward - sv_null)));
    return prob;
}
</script>

In [40]:
%%HTML
<script>
//# make the empty array to build into.
alpha = 1.1;
beta = 0.5;
prob = 0.5;
amb = 0;
prob_levels = [0.13, 0.25, 0.38, 0.50, 0.75];
amb_levels = [0.24, 0.50, 0.74];
max_lott = 50.0;
min_lott = -50.0;
sv = [];
lott_SV = function(money, alpha, beta, prob, amb){
    (prob - beta * amb / 2) * Math.pow(money, alpha)
}



preexisting = random.randint(1, 51, 72);
//# add values to the array with the push() function.
preexisting.forEach(function(money){sv.push((prob - beta * amb / 2) * Math.pow(money, alpha))});
print(sv);

</script>

In [36]:
%%HTML
<script>
//#converts from sv to safe money space
sv_to_safe = function(sv, alpha) {return Math.pow(sv, 1/alpha)}

</script>

In [ ]:
//#compute SV of maximum lottery gain at each probability level, given alpha and beta

